# Trial Randomization: Illusory Pitch

In [ ]:
import json
import numpy as np
from random import shuffle
from itertools import product

OCTAVES = ['3', '5']
OFFSETS = ['-15', '0', '15']
SHIFTS = ['+', '-']

NSESS = 300  # Number of sessions per group
NBLOCKS = 4  # Number of blocks per session

In [ ]:
# Generate a block of 60 trials with a set octave
def randomize_block(octave):
    
    global SHIFTS, OFFSETS
    
    conditions = [c for c in product(octave, SHIFTS, OFFSETS)]
    trials = []
    
    for i in range(0, 10):
        trials += conditions
        
    shuffle(trials)   

    return trials

# Generate Trial Schedules

In [ ]:
sessions = []
for i in range(NSESS):
    
    # Randomize which octave occurs first, then double for ABAB pattern
    octave_order = ['3', '5']
    shuffle(octave_order)
    octave_order += octave_order
    
    # Generate four blocks for the session
    blocks = []
    for i in range(NBLOCKS):
        new_block = randomize_block(octave_order[i])
        blocks.append(new_block)
    session = np.concatenate(blocks)
    session = session.reshape((4, len(OFFSETS) * len(SHIFTS) * 10, 3))
    
    sessions.append(session)

### Sanity Checks

In [ ]:
for i, session in enumerate(sessions):
    
    # Verify session length
    if len(session) != 4:
        raise ValueError('Session %i has improper length!' % i)
    else:
        for i in range(4):
            if len(session[i]) != 60:
                raise ValueError('Session %i has improper length!' % i)
    
    # Verify condition counts
    for octave in OCTAVES:
        oct_mask = session[:, :, 0] == octave
        for shift in SHIFTS:
            shift_mask = session[:, :, 1] == shift
            for offset in OFFSETS:
                offset_mask = session[:, :, 2] == offset
                trials_of_type = np.sum(oct_mask & shift_mask & offset_mask)
                if trials_of_type != 20:
                    raise ValueError('Session %i has %i trials of octave %s, shift %s, and offset %s!' % 
                                     (i, trials_of_type, octave, shift, offset))

print('All sessions verified successfully!')

### Save schedules

In [ ]:
# Shuffle order of sessions and save
shuffle(sessions)
for i, session in enumerate(sessions):
    with open('../schedules/session%i.json' % (i+1), 'w') as f:
        json.dump(session.tolist(), f)